In [121]:
import math
import geopy.distance
from dis import dis
import math
import geopandas as gpd
import numpy
from shapely import wkt
from shapely import wkb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim

%matplotlib inline

train_data = pd.read_csv('../data/stores_train.csv')
test_data = pd.read_csv('../data/stores_test.csv')
busstops = pd.read_csv('../data/busstops_norway.csv')
grunnkrets_age = pd.read_csv('../data/grunnkrets_age_distribution.csv')
grunnkrets_households = pd.read_csv('../data/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('../data/grunnkrets_income_households.csv')
grunnkrets_stripped = pd.read_csv('../data/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('../data/plaace_hierarchy.csv')

In [122]:
# Replace NaN in mall_name and chain_name columns with 'No mall' and 'No chain'
train_data.mall_name = train_data.mall_name.fillna('No mall')
train_data.chain_name = train_data.chain_name.fillna('No chain')
# For test
test_data.mall_name = test_data.mall_name.fillna('No mall')
test_data.chain_name = test_data.chain_name.fillna('No chain')

# Dummy variable for mall or no mall
train_data.loc[train_data['mall_name'].str.contains("No mall", na=False),'mall_dummy'] = 0
train_data.loc[~(train_data['mall_name'].str.contains("No mall", na=False)),'mall_dummy'] = 1
train_data.drop(['mall_name'],axis=1, inplace=True)
# For test
test_data.loc[test_data['mall_name'].str.contains("No mall", na=False),'mall_dummy'] = 0
test_data.loc[~(test_data['mall_name'].str.contains("No mall", na=False)),'mall_dummy'] = 1
test_data.drop(['mall_name'],axis=1, inplace=True)

# 'store_name', 'year', 'sales_channel_name', 'address' columns are redundant, remove them
train_data = train_data.drop('store_name',axis=1)
train_data = train_data.drop('sales_channel_name',axis=1)
train_data = train_data.drop('address',axis=1)
# For test
test_data = test_data.drop('store_name',axis=1)
test_data = test_data.drop('sales_channel_name',axis=1)
test_data = test_data.drop('address',axis=1)

In [123]:
# Add municipality names
train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'municipality_name']], on='grunnkrets_id', how='left')
# we get a bunch of duplicates of store_ids...? Remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')
train_data.municipality_name = train_data.municipality_name.fillna('No municipality name')

# For test
test_data = pd.merge(test_data, grunnkrets_stripped[['grunnkrets_id', 'municipality_name']], on='grunnkrets_id', how='left')
# we get a bunch of duplicates of store_ids...? Remove them.
test_data = test_data.drop_duplicates(subset=['store_id'], keep='first')
test_data.municipality_name = test_data.municipality_name.fillna('No municipality name')

In [124]:
# Total nbr people in each grunnkrets
grunnkrets_age = grunnkrets_age.drop_duplicates(subset=['grunnkrets_id'], keep='last') # if there is value for 2016 we keep it, otherwise 2015
grunnkrets_age = grunnkrets_age.fillna(0)
grunnkrets_age = grunnkrets_age.drop('year',axis=1)
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(str)
grunnkrets_age['total_nbr_people'] = grunnkrets_age.sum(axis=1) # total number of inhabitants
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(int)
train_data = pd.merge(train_data, grunnkrets_age[['grunnkrets_id', 'total_nbr_people']], on='grunnkrets_id', how='left')

/var/folders/v4/l760qmd51vl3yg0k0ly6n5fw0000gn/T/ipykernel_18831/2693807438.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  grunnkrets_age['total_nbr_people'] = grunnkrets_age.sum(axis=1) # total number of inhabitants


In [125]:
######### MUNICIPALITY SIZE GROUPS #########

In [126]:
# Total nbr people in each municipality
municipalities = train_data[["municipality_name", "total_nbr_people"]].groupby(
    ["municipality_name"]
).sum().reset_index()
municipalities = municipalities.rename(columns={'total_nbr_people':'nbr_people_in_municipality'})

# Print distribution to check relevant division into small/medium/large municipality
municipalities = municipalities[municipalities['municipality_name'] != 'No municipality name'] # remove No municipality name (NaN)

#print(municipalities['nbr_people_in_municipality'].describe())
#ax = municipalities.plot.bar(x='municipality_name', y='nbr_people_in_municipality', rot=0)
#print(municipalities)

# Make new column in municipalities for municipality size category, assign categories
conditions = [
    (municipalities['nbr_people_in_municipality'] < 1.612750e+03),
    (municipalities['nbr_people_in_municipality'] >= 1.612750e+03) & (municipalities['nbr_people_in_municipality'] < 5.731000e+03),
    (municipalities['nbr_people_in_municipality'] >= 5.731000e+03) & (municipalities['nbr_people_in_municipality'] < 1.717325e+04),
    (municipalities['nbr_people_in_municipality'] >= 1.717325e+04) & (municipalities['nbr_people_in_municipality'] < (2.109973e+06)-1),
    (municipalities['nbr_people_in_municipality'] >= (2.109973e+06)-1),
]
values = ['1', '2', '3', '4', '0']
municipalities['municipality_size_group'] = np.select(conditions, values)
#print(municipalities)
# municipalities['municipality_size_group'].value_counts() # four size categories of 102-103 municipalities in each, category 0 is the 'No municipality name' one

# merge to train data
train_data = pd.merge(train_data, municipalities[['municipality_name', 'municipality_size_group']], on='municipality_name', how='outer')

# merge to test data
test_data = pd.merge(test_data, municipalities[['municipality_name', 'municipality_size_group']], on='municipality_name', how='outer')


In [127]:
# mean rev per municipality size group
mean_rev_munic = train_data[["municipality_size_group", "revenue"]].groupby(
    ["municipality_size_group"]
).mean().reset_index()
mean_rev_munic = mean_rev_munic.rename(columns={'revenue':'mean_revenue_for_municipality_size_group'})

# merge to train data
train_data = train_data.merge(mean_rev_munic, how="left", on=["municipality_size_group"])
# In case of duplicates, remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')

# merge to test data
test_data = test_data.merge(mean_rev_munic, how="left", on=["municipality_size_group"])
# In case of duplicates, remove them.
test_data = test_data.drop_duplicates(subset=['store_id'], keep='first')

In [128]:
# median rev per municipality size group
median_rev_munic = train_data[["municipality_size_group", "revenue"]].groupby(
    ["municipality_size_group"]
).median().reset_index()
median_rev_munic = median_rev_munic.rename(columns={'revenue':'median_revenue_for_municipality_size_group'})

# merge to train data
train_data = train_data.merge(median_rev_munic, how="left", on=["municipality_size_group"])
# In case of duplicates, remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')

# merge to test data
test_data = test_data.merge(median_rev_munic, how="left", on=["municipality_size_group"])
# In case of duplicates, remove them.
test_data = test_data.drop_duplicates(subset=['store_id'], keep='first')

In [129]:
# st dev per municipality size group
stdev_munic = train_data[["municipality_size_group", "revenue"]].groupby(
    ["municipality_size_group"]
).std().reset_index()
stdev_munic = stdev_munic.rename(columns={'revenue':'st_dev_of_revenue_for_municipality_size_group'})

# merge to train data
train_data = train_data.merge(stdev_munic, how="left", on=["municipality_size_group"])
# In case of duplicates, remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')

# merge to test data
test_data = test_data.merge(stdev_munic, how="left", on=["municipality_size_group"])
# In case of duplicates, remove them.
test_data = test_data.drop_duplicates(subset=['store_id'], keep='first')

In [130]:
######### MUNICIPALITY REVENUE GROUPS #########

In [131]:
train_data = pd.merge(train_data, plaace_hierarchy[['plaace_hierarchy_id', 'lv1', 'lv2', 'lv3']], on='plaace_hierarchy_id', how='outer')
train_data['lv1']= train_data['lv1'].astype('category')
train_data['lv2']= train_data['lv2'].astype('category')
train_data['lv3']= train_data['lv3'].astype('category')

# For test
test_data = pd.merge(test_data, plaace_hierarchy[['plaace_hierarchy_id', 'lv1', 'lv2', 'lv3']], on='plaace_hierarchy_id', how='outer')
test_data['lv1']= test_data['lv1'].astype('category')
test_data['lv2']= test_data['lv2'].astype('category')
test_data['lv3']= test_data['lv3'].astype('category')

In [132]:
# Mean revenue for municipality and lv2
municipalities_rev = train_data[["municipality_name", "lv2", "revenue"]].groupby(
    ["municipality_name", "lv2"]
).mean().reset_index()
municipalities_rev = municipalities_rev.rename(columns={'revenue':'mean_revenue_for_municipality_and_level2'})


# Print distribution to check relevant division into low/medium/high revenue municipality
municipalities_rev = municipalities_rev[municipalities_rev['municipality_name'] != 'No municipality name'] # remove No municipality name (NaN)

#print(municipalities_rev['mean_revenue_for_municipality_and_level2'].describe())
#ax = municipalities_rev.plot.bar(x='municipality_name', y='mean_revenue_for_municipality_and_level2', rot=0)


# Make new column in municipalities_rev for municipality revenue category, assign categories
conds = [
    (municipalities_rev['mean_revenue_for_municipality_and_level2'] < 1.931500),
    (municipalities_rev['mean_revenue_for_municipality_and_level2'] >= 1.931500) & (municipalities_rev['mean_revenue_for_municipality_and_level2'] < 4.154000),
    (municipalities_rev['mean_revenue_for_municipality_and_level2'] >= 4.154000) & (municipalities_rev['mean_revenue_for_municipality_and_level2'] < 8.806000),
    (municipalities_rev['mean_revenue_for_municipality_and_level2'] >= 8.806000),
]
vals = ['1', '2', '3', '4']
municipalities_rev['municipality_rev_group_lv2'] = np.select(conds, vals)

#print(municipalities_rev)
#municipalities_rev['municipality_rev_group_lv2'].value_counts() # Varför en grupp med 0?????

# merge to train data
train_data = pd.merge(train_data, municipalities_rev[['municipality_name', 'municipality_rev_group_lv2']], on='municipality_name', how='outer')

# merge to test data
test_data = pd.merge(test_data, municipalities_rev[['municipality_name', 'municipality_rev_group_lv2']], on='municipality_name', how='outer')

municipalities_rev['municipality_rev_group_lv2'].value_counts() # Varför en grupp med 0?????

0    5091
4     778
1     778
3     777
2     776
Name: municipality_rev_group_lv2, dtype: int64

In [133]:
# mean rev per municipality rev group lv2
mean_rev_munic_lv2 = train_data[["municipality_rev_group_lv2", "revenue"]].groupby(
    ["municipality_rev_group_lv2"]
).mean().reset_index()
mean_rev_munic_lv2 = mean_rev_munic_lv2.rename(columns={'revenue':'mean_revenue_for_municipality_rev_group_lv2'})

# merge to train data
train_data = train_data.merge(mean_rev_munic_lv2, how="left", on=["municipality_rev_group_lv2"])
# In case of duplicates, remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')

# merge to test data
test_data = test_data.merge(mean_rev_munic_lv2, how="left", on=["municipality_rev_group_lv2"])
# In case of duplicates, remove them.
test_data = test_data.drop_duplicates(subset=['store_id'], keep='first')

In [134]:
# Drop groups
train_data = train_data.drop('municipality_size_group',axis=1)
train_data = train_data.drop('municipality_rev_group_lv2',axis=1)

In [135]:
train_data.head()

,store_id,year,plaace_hierarchy_id,grunnkrets_id,lat,lon,chain_name,revenue,mall_dummy,municipality_name,total_nbr_people,mean_revenue_for_municipality_size_group,median_revenue_for_municipality_size_group,st_dev_of_revenue_for_municipality_size_group,lv1,lv2,lv3,mean_revenue_for_municipality_rev_group_lv2
0,983540538-974187930-44774,2016.0,1.1.1.0,6020303.0,59.743104,10.204928,MCDONALDS,17.998,1.0,Drammen,157.0,8.461341,3.538,16.221455,1,1.1,1.1.1,8.329332
20,999125026-999139256-472734,2016.0,1.1.2.0,6020509.0,59.742529,10.157972,No chain,4.395,1.0,Drammen,126.0,8.461341,3.538,16.221455,1,1.1,1.1.2,8.329332
40,898083292-898135322-538507,2016.0,1.1.2.0,6020604.0,59.738200,10.203061,FUJI SUSHI,7.103,0.0,Drammen,1102.0,8.461341,3.538,16.221455,1,1.1,1.1.2,8.329332
60,916744765-916758723-821461,2016.0,1.1.2.0,6020602.0,59.738563,10.192774,No chain,1.051,0.0,Drammen,1035.0,8.461341,3.538,16.221455,1,1.1,1.1.2,8.329332
80,916139195-998469740-540508,2016.0,1.1.2.0,6020303.0,59.744230,10.206481,No chain,3.814,0.0,Drammen,157.0,8.461341,3.538,16.221455,1,1.1,1.1.2,8.329332


In [136]:

# antal folk => stor/medel/liten by - size groups DONE
# mean rev per municipality size group DONE
# st dev rev per municipality DONE
# median rev per municipality size group DONE

# gruppera kommuner på mean rev => high revenue municipalities/medium revenue municipalities/low revenue municipalities
# mean rev per municipality rev group and lv1
# mean rev per municipality rev group and lv2 DONE
# mean rev per municipality rev group and lv3
# median rev per municipality rev group and lv1
# median rev per municipality rev group and lv2
# median rev per municipality rev group and lv3
# st dev rev per municipality rev group and lv1
# st dev rev per municipality rev group and lv2
# st dev rev per municipality rev group and lv3
